In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')

In [3]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X_train = train.drop(['label'], axis=1)
y_train = train['label']

In [5]:
X_train.shape

(42000, 784)

In [6]:
y_train.shape

(42000,)

In [7]:
# 欠損値確認
X_train.isna().any().describe()

count       784
unique        1
top       False
freq        784
dtype: object

In [8]:
test.isna().any().describe()

count       784
unique        1
top       False
freq        784
dtype: object

In [9]:
X_train /= 255.0
test /= 255.0

In [10]:
X_train = X_train.values.reshape(-1, 28, 28, 1)
test = test.values.reshape(-1, 28, 28, 1)

In [11]:
X_train.shape

(42000, 28, 28, 1)

In [12]:
from keras.utils import to_categorical
y_train = to_categorical(y_train, num_classes=10)

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_validate, y_train, y_validate = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [14]:
X_train.shape

(33600, 28, 28, 1)

In [15]:
y_train.shape

(33600, 10)

In [16]:
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

Metal device set to: Apple M2

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



In [17]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 128)         73856 

In [28]:
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

In [29]:
epochs = 3
batch_size = 64

In [30]:
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_validate, y_validate))

Epoch 1/3
525/525 [==============================] - 8s 13ms/step - loss: 0.0375 - accuracy: 0.9880 - val_loss: 0.0399 - val_accuracy: 0.9886
Epoch 2/3
525/525 [==============================] - 7s 13ms/step - loss: 0.0279 - accuracy: 0.9918 - val_loss: 0.0475 - val_accuracy: 0.9870
Epoch 3/3
525/525 [==============================] - 7s 13ms/step - loss: 0.0217 - accuracy: 0.9930 - val_loss: 0.0467 - val_accuracy: 0.9863


In [31]:
history.history

{'loss': [0.037486422806978226, 0.02792011760175228, 0.021715274080634117],
 'accuracy': [0.9880357384681702, 0.9918452501296997, 0.992976188659668],
 'val_loss': [0.03994480147957802, 0.04753310978412628, 0.04669208079576492],
 'val_accuracy': [0.9885714054107666, 0.9870238304138184, 0.9863095283508301]}

In [32]:
results = model.predict(test)
results

875/875 [==============================] - 2s 2ms/step


array([[7.52066395e-11, 3.53867347e-15, 1.00000000e+00, ...,
        5.09683273e-09, 1.58976911e-11, 4.55738637e-13],
       [9.99999881e-01, 6.07428037e-19, 8.80361384e-09, ...,
        3.29841576e-09, 5.82459862e-12, 3.06450474e-08],
       [3.28791486e-12, 1.45679689e-14, 1.42279183e-11, ...,
        2.42964777e-08, 3.11049071e-08, 9.99999881e-01],
       ...,
       [2.75991943e-15, 2.21004830e-15, 4.33412507e-12, ...,
        8.52510562e-10, 8.44291803e-10, 3.83936286e-11],
       [1.45663397e-12, 2.68191013e-16, 1.30783556e-14, ...,
        8.49364099e-08, 1.25395597e-10, 9.99999642e-01],
       [1.23515965e-12, 6.28861083e-17, 1.00000000e+00, ...,
        7.30566996e-12, 4.11136875e-10, 8.89154608e-14]], dtype=float32)

In [33]:
results = np.array(results)
results

array([[7.52066395e-11, 3.53867347e-15, 1.00000000e+00, ...,
        5.09683273e-09, 1.58976911e-11, 4.55738637e-13],
       [9.99999881e-01, 6.07428037e-19, 8.80361384e-09, ...,
        3.29841576e-09, 5.82459862e-12, 3.06450474e-08],
       [3.28791486e-12, 1.45679689e-14, 1.42279183e-11, ...,
        2.42964777e-08, 3.11049071e-08, 9.99999881e-01],
       ...,
       [2.75991943e-15, 2.21004830e-15, 4.33412507e-12, ...,
        8.52510562e-10, 8.44291803e-10, 3.83936286e-11],
       [1.45663397e-12, 2.68191013e-16, 1.30783556e-14, ...,
        8.49364099e-08, 1.25395597e-10, 9.99999642e-01],
       [1.23515965e-12, 6.28861083e-17, 1.00000000e+00, ...,
        7.30566996e-12, 4.11136875e-10, 8.89154608e-14]], dtype=float32)

In [34]:
results = np.argmax(results, axis=1)

In [35]:
results = pd.Series(results)

In [36]:
submission_csv= pd.DataFrame({'ImageId': range(1, 28001), 'Label': results})
submission_csv.to_csv('output/submission.csv', index=False)